In [1]:
import pandas as pd
from keras.utils import to_categorical

Using TensorFlow backend.


In [3]:

New_df=pd.read_csv("train.csv")
New_df.drop("ID_code",axis=1,inplace=True)
X=New_df.drop("target",axis=1)
y=New_df["target"]
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=2)
X_train_res, y_train_res = sm.fit_sample(X, y.ravel())
from keras.utils import to_categorical
y_train_res=to_categorical(y_train_res)


print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res==1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res==0)))


After OverSampling, the shape of train_X: (359804, 200)
After OverSampling, the shape of train_y: (359804, 2) 

After OverSampling, counts of label '1': [179902 179902]
After OverSampling, counts of label '0': [179902 179902]


In [4]:
import tensorflow as tf
import keras.backend as b
b.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

# For Finding Random Parameters

In [50]:
New_df=pd.read_csv("Stockprediction82.csv")
New_df.drop("Unnamed: 0",axis=1,inplace=True)
target_0=New_df[New_df["target"]==0]
target_1=New_df[New_df["target"]==1]
target_0=target_0[:5000]
target_1=target_1[:5000]
X=target_0.append(target_1)
print(len(X))
y=X["target"]
X.drop("target",axis=1,inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)


X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

10000


In [4]:
import math
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Activation,BatchNormalization
from keras import losses
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras import regularizers

def Randomized_Model(lr=0.0001,dropout=0.5,optimizer='Adam',loss='mean_squared_error',
                    last_activation="softmax",activation="relu",clipnorm=0.1,
                    decay=1e-2,momentum=0.5
                    ):
       
    
    
    

    
    #setting up loss fucntions
    loss=losses.mean_squared_error
    if(loss=='mean_squared_error'):
        loss=losses.mean_squared_error
    if(loss=="poisson"):
        loss=keras.losses.poisson
    if(loss=="mean_absolute_error"):
        loss=keras.losses.mean_absolute_percentage_error
    if(loss=="mean_squared_logarithmic_error"):
        loss=keras.losses.mean_squared_logarithmic_error
    if(loss=="binary_crossentropy"):
        loss=keras.losses.binary_crossentropy
    if(loss=="hinge"):
        loss=keras.losses.hinge
        
    #setting up Optimizers
    opt=keras.optimizers.Adam(lr=lr, decay=decay, beta_1=0.9, beta_2=0.999)
    if optimizer=="Adam":
        opt=keras.optimizers.Adam(lr=lr, decay=decay, beta_1=0.9, beta_2=0.999)
    if optimizer=="Adagrad":
        opt=keras.optimizers.Adagrad(lr=lr, epsilon=None, decay=decay)
    if optimizer=="sgd":
        opt=keras.optimizers.SGD(lr=lr, momentum=momentum, decay=decay, nesterov=False)
    if optimizer=="RMSprop":
        opt=keras.optimizers.RMSprop(lr=lr, rho=0.9, epsilon=None, decay=0.0)
    if optimizer=="Adamax":
        opt=keras.optimizers.Adamax(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
        
        
    #last activation layer
    if last_activation=="softmax":
        last_activation=keras.activations.softmax
    if last_activation=="sigmoid":
        last_activation=keras.activations.sigmoid
    
    
    
    
    
    model=Sequential()
    
    
    
    

    model.add(Dense(units=84,activation=activation))
    model.add(Dropout(dropout))
    

    model.add(Dense(units=62,activation=activation))
    model.add(Dropout(dropout))
    

    model.add(Dense(units=31,activation=activation))
    model.add(Dropout(dropout))
    


    model.add(Dense(units=16,activation=activation))
    model.add(Dropout(dropout))
    


    model.add(Dense(units=16,activation=activation))
    model.add(Dropout(dropout))
    


    model.add(Dense(units=2,activation=last_activation))

    model.compile(loss=loss,optimizer=opt,
                     metrics=['accuracy'])
    
    
    
    

    return model




In [52]:
params = {'lr': (0.0001, 0.0009,0.0002 ,0.001),
     'epochs': [20],
     'dropout': (0.1, 0.2,0.3),
     'optimizer': ['Adam','Adagrad','sgd','RMSprop','Adamax'],
     'loss': ["binary_crossentropy","mean_squared_error","hinge","mean_absolute_error","mean_squared_logarithmic_error","poisson"],
     'last_activation': ["softmax","sigmoid"],
     'activation' :["relu","selu","linear","sigmoid"],
     'clipnorm':(0.0,0.5,1),
     'decay':(1e-6,1e-4,1e-8),
     'momentum':(0.9,0.5,0.2),
     }

In [54]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer
# model class to use in the scikit random search CV 

model = KerasClassifier(build_fn=Randomized_Model, epochs=10, batch_size=10, verbose=1)
RandomizedSearchfit = RandomizedSearchCV(estimator=model, cv=KFold(10), param_distributions=params, 
                          verbose=1,  n_iter=10, n_jobs=1)
RandomizedSearchfit.fit(X_train, y_train)


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Epoch 1/20
6300/6300 [==============================] - 3s 549us/step - loss: 0.4954 - acc: 0.5071
Epoch 2/20
6300/6300 [==============================] - 2s 371us/step - loss: 0.4898 - acc: 0.5024
Epoch 3/20
6300/6300 [==============================] - 2s 365us/step - loss: 0.4955 - acc: 0.5059
Epoch 4/20
6300/6300 [==============================] - 2s 288us/step - loss: 0.4969 - acc: 0.5013
Epoch 5/20
6300/6300 [==============================] - 2s 377us/step - loss: 0.5002 - acc: 0.5016
Epoch 6/20
6300/6300 [==============================] - 2s 368us/step - loss: 0.4978 - acc: 0.5016
Epoch 7/20
6300/6300 [==============================] - 2s 377us/step - loss: 0.4984 - acc: 0.5016
Epoch 8/20
6300/6300 [==============================] - 2s 381us/step - loss: 0.4983 - acc: 0.5016
Epoch 9/20
6300/6300 [==============================] - 2s 377us/step - loss: 0.4983 - acc: 0.5016
Epoch 10/20
6300/6300 [==============================] - 2s 378us/step - loss: 0.4984 - acc: 0.5016
Epoch 11/

[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed: 118.8min finished


Epoch 1/20
7000/7000 [==============================] - 21s 3ms/step - loss: 0.2748 - acc: 0.5113
Epoch 2/20
7000/7000 [==============================] - 4s 602us/step - loss: 0.2420 - acc: 0.5749
Epoch 3/20
7000/7000 [==============================] - 4s 602us/step - loss: 0.2246 - acc: 0.6339
Epoch 4/20
7000/7000 [==============================] - 4s 609us/step - loss: 0.2144 - acc: 0.6603
Epoch 5/20
7000/7000 [==============================] - 4s 610us/step - loss: 0.2075 - acc: 0.6793
Epoch 6/20
7000/7000 [==============================] - 4s 610us/step - loss: 0.2045 - acc: 0.6821
Epoch 7/20
7000/7000 [==============================] - 4s 606us/step - loss: 0.2029 - acc: 0.6877
Epoch 8/20
7000/7000 [==============================] - 4s 611us/step - loss: 0.2008 - acc: 0.6883
Epoch 9/20
7000/7000 [==============================] - 4s 611us/step - loss: 0.1992 - acc: 0.6946
Epoch 10/20
7000/7000 [==============================] - 4s 603us/step - loss: 0.1976 - acc: 0.6939
Epoch 11/2

RandomizedSearchCV(cv=KFold(n_splits=10, random_state=None, shuffle=False),
          error_score='raise-deprecating',
          estimator=<keras.wrappers.scikit_learn.KerasClassifier object at 0x7fdfe0b985c0>,
          fit_params=None, iid='warn', n_iter=10, n_jobs=1,
          param_distributions={'lr': (0.0001, 0.0009, 0.0002, 0.001), 'epochs': [20], 'dropout': (0.1, 0.2, 0.3), 'optimizer': ['Adam', 'Adagrad', 'sgd', 'RMSprop', 'Adamax'], 'loss': ['binary_crossentropy', 'mean_squared_error', 'hinge', 'mean_absolute_error', 'mean_squared_logarithmic_error', 'poisson'], 'last_activation': ['softmax', 'sigmoid'], 'activation': ['relu', 'selu', 'linear', 'sigmoid'], 'clipnorm': (0.0, 0.5, 1), 'decay': (1e-06, 0.0001, 1e-08), 'momentum': (0.9, 0.5, 0.2)},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=1)

In [58]:
best=RandomizedSearchfit.best_params_
best

{'optimizer': 'Adam',
 'momentum': 0.2,
 'lr': 0.001,
 'loss': 'mean_squared_error',
 'last_activation': 'softmax',
 'epochs': 20,
 'dropout': 0.1,
 'decay': 1e-06,
 'clipnorm': 0.5,
 'activation': 'relu'}

# Model with best Parameters

In [14]:
import math
import keras
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold
from sklearn.metrics import make_scorer
from keras.models import Sequential,Model
from keras.layers import Dense,Dropout,Activation,BatchNormalization
from keras import losses
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras import regularizers

def best_parameter_model():
    momentum=0.2  
    decay=0
    lr=0.0001 
    dropout=0.2
    optimizer='Adam'
    loss='mean_squared_error' 
    last_activation='softmax'
    activation='relu'
    clipnorm=1 
    l2=0.01 
    
    '''
    momentum=best["momentum"]
    decay=best["decay"]
    lr=best["lr"] 
    dropout=best["dropout"]
    optimizer=best["optimizer"]
    loss=best["loss"]
    last_activation=best["last_activation"]
    activation=best["activation"]
    clipnorm=best["clipnorm"]
    l2=best["l2"]
    '''
    #setting up loss fucntions
    loss=losses.mean_squared_error
    if(loss=='mean_squared_error'):
        loss=losses.mean_squared_error
    if(loss=="poisson"):
        loss=keras.losses.poisson
    if(loss=="mean_absolute_error"):
        loss=keras.losses.mean_absolute_percentage_error
    if(loss=="mean_squared_logarithmic_error"):
        loss=keras.losses.mean_squared_logarithmic_error
    if(loss=="binary_crossentropy"):
        loss=keras.losses.binary_crossentropy
    if(loss=="hinge"):
        loss=keras.losses.hinge
        
    #setting up Optimizers
    opt=keras.optimizers.Adam(lr=lr, decay=decay, beta_1=0.9, beta_2=0.999)
    if optimizer=="Adam":
        opt=keras.optimizers.Adam(lr=lr, decay=decay, beta_1=0.9, beta_2=0.999)
    if optimizer=="Adagrad":
        opt=keras.optimizers.Adagrad(lr=lr, epsilon=None, decay=decay)
    if optimizer=="sgd":
        opt=keras.optimizers.SGD(lr=lr, momentum=momentum, decay=decay, nesterov=False)
    if optimizer=="RMSprop":
        opt=keras.optimizers.RMSprop(lr=lr, rho=0.9, epsilon=None, decay=0.0)
    if optimizer=="Adamax":
        opt=keras.optimizers.Adamax(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0)
        
        
    #last activation layer
    if last_activation=="softmax":
        last_activation=keras.activations.softmax
    if last_activation=="sigmoid":
        last_activation=keras.activations.sigmoid
    
    
    
    
    
    model=Sequential()
    
    
    
    model.add(Dense(units=128,activation=activation))
    model.add(Dropout(dropout))

    model.add(Dense(units=86,activation=activation))
    model.add(Dropout(dropout))
    

    model.add(Dense(units=64,activation=activation))
    model.add(Dropout(dropout))
    

    model.add(Dense(units=32,activation=activation))
    model.add(Dropout(dropout))
    


    model.add(Dense(units=16,activation=activation))
    model.add(Dropout(dropout))
    


    model.add(Dense(units=16,activation=activation))
    model.add(Dropout(dropout))
   


    model.add(Dense(units=2,activation=last_activation))

    model.compile(loss=loss,optimizer=opt,
                     metrics=['accuracy'])
    
    model.compile(loss=loss ,optimizer=opt,
                 metrics=['accuracy'])
    
    

    return model




In [ ]:
import matplotlib.pyplot as plt
model=best_parameter_model()
history=model.fit(X_train_res, y_train_res,batch_size=50,epochs=500)





Epoch 1/500
359804/359804 [==============================] - 29s 79us/step - loss: 0.2297 - acc: 0.6304
Epoch 2/500
359804/359804 [==============================] - 27s 75us/step - loss: 0.1675 - acc: 0.7596
Epoch 3/500
359804/359804 [==============================] - 29s 80us/step - loss: 0.1498 - acc: 0.7888
Epoch 4/500
359804/359804 [==============================] - 29s 82us/step - loss: 0.1389 - acc: 0.8065
Epoch 5/500
359804/359804 [==============================] - 29s 81us/step - loss: 0.1319 - acc: 0.8169
Epoch 6/500
359804/359804 [==============================] - 29s 82us/step - loss: 0.1259 - acc: 0.8266
Epoch 7/500
359804/359804 [==============================] - 29s 80us/step - loss: 0.1214 - acc: 0.8340
Epoch 8/500
359804/359804 [==============================] - 29s 79us/step - loss: 0.1174 - acc: 0.8400
Epoch 9/500
359804/359804 [==============================] - 29s 80us/step - loss: 0.1143 - acc: 0.8446
Epoch 10/500
359804/359804 [==============================] - 28

(252032, 82)

# Traning on whole dataset

# 15 coloums 52% accuracy
# 82 coloums 62% accuracy

In [34]:
df_train=pd.read_csv("train.csv")
df_train.drop("ID_code",axis=1,inplace=True)
X=df_train.drop("target",axis=1)
y=df_train["target"]

In [ ]:
import numpy as np
X=np.asarray(X)
y=np.asarray(y)
#y=to_categorical(y)

model=best_parameter_model()
model=best_parameter_model()
history=model.fit(X_train_res, y_train_res,batch_size=100,epochs=200)




Epoch 1/200
252032/252032 [==============================] - 11s 44us/step - loss: 0.2348 - acc: 0.6082
Epoch 2/200
252032/252032 [==============================] - 10s 40us/step - loss: 0.1976 - acc: 0.7013
Epoch 3/200
252032/252032 [==============================] - 10s 40us/step - loss: 0.1868 - acc: 0.7224
Epoch 4/200
252032/252032 [==============================] - 10s 41us/step - loss: 0.1808 - acc: 0.7335
Epoch 5/200
252032/252032 [==============================] - 10s 40us/step - loss: 0.1757 - acc: 0.7420
Epoch 6/200
252032/252032 [==============================] - 10s 40us/step - loss: 0.1714 - acc: 0.7497
Epoch 7/200
252032/252032 [==============================] - 10s 40us/step - loss: 0.1685 - acc: 0.7548
Epoch 8/200
252032/252032 [==============================] - 10s 40us/step - loss: 0.1654 - acc: 0.7593
Epoch 9/200
252032/252032 [==============================] - 10s 40us/step - loss: 0.1630 - acc: 0.7644
Epoch 10/200
252032/252032 [==============================] - 10

# Submission Part

In [10]:
test=pd.read_csv("test.csv")
test_id=test["ID_code"]
test=test.drop("ID_code",axis=1)

#test_coloums=New_df.columns.values[1:]

#test=test[test_coloums]

print("Test shape",test.shape)

target=model.predict(test)
import numpy as np
prediction=[]
for i in target:
    a=np.argmax(i)
    prediction.append(a)
len(prediction)

Test shape (200000, 200)


200000

In [12]:

print("Saving submission file")
sample = pd.DataFrame()
sample["ID_code"] = test_id.values
sample["target"] = prediction
sample.to_csv('submission.csv', index=False)

Saving submission file
